# 2. Get & Set Human Entity Rule

## Get Human Entity Rule

In [1]:
import pandas as pd

In [2]:
# Import dataset
entityDF10Juz = pd.read_excel('./Dataset/human-entity-10-juz.xlsx')
pd.set_option('display.max_rows', 999999)

In [3]:
def findHumanEntityPatterns(file):
    
    tempPatterns = [[]]
    finalPatterns = []
    currentTag = 0
    opentagCount = 0
    df = pd.DataFrame({'pattern': []})
    
    for i in range(len(file)):

        closetags = [char for char in str(file.at[i, "closetag"])]
        opentags = [char for char in str(file.at[i, "opentag"])]

        for j in range(len(opentags)):
            if opentags[j] == '(':
                opentagCount += 1

        if opentagCount > 0:
            tempPatterns[currentTag].append(file.at[i, "tag"])

        for k in range(len(closetags)):
            if closetags[k] == ')':
                if opentagCount == 1:
                    toInput = (str(tempPatterns[currentTag]))
                    df = df.append({'pattern' : toInput}, ignore_index=True)
                    finalPatterns.append(tempPatterns[currentTag])
                    currentTag += 1
                    tempPatterns.append([])
                opentagCount -= 1
                
    if tempPatterns[len(tempPatterns)-1] == []:
        del tempPatterns[len(tempPatterns)-1]

    
    tempPatterns = set(tuple(element) for element in tempPatterns)
    tempPatterns = [list(t) for t in set(tuple(element) for element in tempPatterns)]    
        
    return tempPatterns

In [4]:
patterns10Juz = findHumanEntityPatterns(entityDF10Juz)

## Set Human Entity Rule

In [5]:
# Import data
quranicArab = pd.read_excel('./Output/quranic-corpus-morphology-0.4-with-arab-juz-2.xlsx', encoding='utf8')
pd.set_option('display.max_rows', 999999) # Untuk menampilkan semua data sekaligus

In [6]:
# Prepare data
def convertAndGroupToArray(data):
    
    ayats = [[]]

    for index, row in data.iterrows():
        location = row['LOCATION']
        ayat = location.split(':')[1]
        data.at[index, 'LOCATION'] = ayat

    length = len(data)
    i = 0
    data.at[0, 'LOCATION'] = 0
    
    for index, row in data.iterrows():
        
        # tokenizing location
        if index < length-1:
            currentLocation = row['LOCATION']
            nextLocation = data.at[index+1, 'LOCATION']

            if index == length-2 or currentLocation != nextLocation:
                i = i+1
                ayats.append([])
            
            data.at[index, 'LOCATION'] = i
            
            words = {
                'OPEN TAG' : '',
                'AYAT' : int(row['LOCATION']),
                'ARAB' : row['ARAB'],
                'TAG' : row['TAG'],
                'CLOSE TAG' : ''
            }
            
            ayats[i].append(words)
    
    return ayats
     
ayats = convertAndGroupToArray(quranicArab)

### Wihout Numbers

In [7]:
# Rule based logic
def findHumanEntity(ayats):
    
    ayatIndex = 0
    for ayat in ayats:
        for pattern in patterns10Juz:
            patternLength = len(pattern)
            for wordIndex in range(len(ayat) - patternLength):
                # Get set of words based on the pattern length
                wordTagsSet = []
                for wordOfSetIndex in range((patternLength)):
                    wordTagsSet.append(ayat[wordIndex+wordOfSetIndex]['TAG'])
                if str(wordTagsSet) == str(pattern):
                    ayat[wordIndex]['OPEN TAG'] += '('
                    ayat[wordIndex+patternLength]['CLOSE TAG'] += ')'
        ayatIndex += 1
                
findHumanEntity(ayats)

In [13]:
# ayatDF = pd.DataFrame({'AYAT': [], 'OPEN TAG': [], 'ARAB': [], 'TAG': [], 'CLOSE TAG': []});
ayatDF = pd.DataFrame({'OPEN TAG': [], 'ARAB': [], 'TAG': [], 'CLOSE TAG': []});

ayatCollections = []
openTagCollections = []
arabCollections = []
tagCollections = []
closeTagCollections = []

for ayat in ayats:
    for word in ayat:
#         ayatDF = ayatDF.append({'AYAT': word['AYAT'], 'OPEN TAG': word['OPEN TAG'], 'ARAB': word['ARAB'], 'TAG': word['TAG'], 'CLOSE TAG': word['CLOSE TAG']}, ignore_index=True)
        ayatDF = ayatDF.append({'OPEN TAG': word['OPEN TAG'], 'ARAB': word['ARAB'], 'TAG': word['TAG'], 'CLOSE TAG': word['CLOSE TAG']}, ignore_index=True)
        
ayatDF

,OPEN TAG,ARAB,TAG,CLOSE TAG
0,,سَ,FUT,
1,((,يَقُولُ,V,
2,((,ٱل,DET,)
3,(,سُّفَهَاءُ,N,)
4,,مِنَ,P,)))
5,((,ٱل,DET,
6,(,نَّاسِ,N,)
7,,مَا,INTG,))
8,((,وَلَّىٰ,V,
9,,هُمْ,PRON,)


209 entitas dalam 1 halaman

### With Numbers

In [ ]:
# Rule based logic
def findHumanEntity(ayats):
    
    ayatIndex = 142
    entityIndex = 0
    for ayat in ayats:
        for pattern in patterns10Juz:
            patternLength = len(pattern)
            for wordIndex in range(len(ayat) - patternLength):
                # Get set of words based on the pattern length
                wordTagsSet = []
                for wordOfSetIndex in range((patternLength)):
                    wordTagsSet.append(ayat[wordIndex+wordOfSetIndex]['TAG'])
                if str(wordTagsSet) == str(pattern):
                    ayat[wordIndex]['OPEN TAG'] += str(entityIndex) + '('
                    ayat[wordIndex+patternLength]['CLOSE TAG'] += str(entityIndex) + ')'
                    entityIndex += 1
        ayatIndex += 1
                
findHumanEntity(ayats)

In [ ]:
ayatDfNumber = pd.DataFrame({'AYAT': [], 'OPEN TAG': [], 'ARAB': [], 'TAG': [], 'CLOSE TAG': []});

ayatCollections = []
openTagCollections = []
arabCollections = []
tagCollections = []
closeTagCollections = []

for ayat in ayats:
    for word in ayat:
        ayatDfNumber = ayatDfNumber.append({'AYAT': word['AYAT'], 'OPEN TAG': word['OPEN TAG'], 'ARAB': word['ARAB'], 'TAG': word['TAG'], 'CLOSE TAG': word['CLOSE TAG']}, ignore_index=True)
        
ayatDfNumber

### Human Entity Juz 2

In [9]:
data = pd.read_excel('./Dataset/human-entity-juz-2.xlsx')
pd.set_option('display.max_rows', 999999)

In [10]:
import numpy as np

In [11]:
entityJuz2 = data.replace(np.nan, '', regex=True)

In [12]:
entityJuz2[['OPEN TAG','ARAB','TAG','CLOSE TAG']][:]

,OPEN TAG,ARAB,TAG,CLOSE TAG
0,,سَ,FUT,
1,,يَقُولُ,V,
2,,ٱل,DET,
3,(,سُّفَهَاءُ,N,)
4,,مِنَ,P,
5,,ٱل,DET,
6,(,نَّاسِ,N,)
7,,مَا,INTG,
8,,وَلَّىٰ,V,
9,,هُمْ,PRON,


17 entitas manusia dalam 1 halaman

In [14]:
ayatDF

,OPEN TAG,ARAB,TAG,CLOSE TAG
0,,سَ,FUT,
1,((,يَقُولُ,V,
2,((,ٱل,DET,)
3,(,سُّفَهَاءُ,N,)
4,,مِنَ,P,)))
5,((,ٱل,DET,
6,(,نَّاسِ,N,)
7,,مَا,INTG,))
8,((,وَلَّىٰ,V,
9,,هُمْ,PRON,)


In [17]:
entityJuz2[['OPEN TAG','ARAB','TAG','CLOSE TAG']][:5]

,OPEN TAG,ARAB,TAG,CLOSE TAG
0,,سَ,FUT,
1,,يَقُولُ,V,
2,,ٱل,DET,
3,(,سُّفَهَاءُ,N,)
4,,مِنَ,P,


In [21]:
a = entityJuz2.at[0, 'OPEN TAG']
b = entityJuz2.at[0, 'CLOSE TAG']

In [22]:
b = ayatDF.at[0, 'OPEN TAG']

In [35]:
# if entityJuz2.at[0, 'OPEN TAG'] == "" :
#     print('kosong')
# else:
#     print('tidak kosong')

if entityJuz2.at[0, 'OPEN TAG'] != "" :
    print('tidak kosong')
else:
    print('kosong')

kosong


In [51]:
validEntity = 0
for i in range(len(ayatDF.index)):
    if entityJuz2.at[i, 'OPEN TAG'] == ayatDF.at[i, 'OPEN TAG'] and entityJuz2.at[i, 'OPEN TAG'] != "" and ayatDF.at[i, 'OPEN TAG'] != "":
        print('index', i)
        validEntity += 1
        
print(validEntity)

In [52]:
validEntity = 0
for i in range(221):
    if entityJuz2.at[i, 'OPEN TAG'] == ayatDF.at[i, 'OPEN TAG'] and entityJuz2.at[i, 'OPEN TAG'] != "" and ayatDF.at[i, 'OPEN TAG'] != "":
        print('index', i)
        validEntity += 1
        
print(validEntity)

index 3
index 6
index 31
index 43
index 50
index 53
index 71
7
